In [1]:
import os
import copy
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cma

from gymnasium.spaces import Dict, Box, Discrete, MultiDiscrete
from torch import nn as nn

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
class PlantAirControl(gym.Env):
    # Constants are hard-coded for now but can set up to read from a spreadsheet
    LIGHT_HEAT = 1.0

    FAN_MAX_WATTAGE = 0.5 # kW
    HEAT_MAX_WATTAGE = 1.5 # kW
    COOL_MAX_WATTAGE = 1.5 # kW

    FAN_MAX_AIR_INOUT_RATE = 1.0 
    HEAT_MAX_WATER_TEMP_UP_RATE = 1.0
    COOL_MAX_WATER_TEMP_DOWN_RATE = 1.0

    DESIRED_TEMP = [30.0, 40.0] # Celcius
    DESIRED_HUMID = [50.0, 70.0] # Percentage (relavtive humidity)
    DESIRED_VPD = [10.0, 20.0] 

    def __init__(self, render_mode=None,
                fan_max_wattage = FAN_MAX_WATTAGE,
                heat_max_wattage = HEAT_MAX_WATTAGE,
                cool_max_wattage = COOL_MAX_WATTAGE,

                fan_max_air_inout_rate = FAN_MAX_AIR_INOUT_RATE,
                heat_max_water_temp_up_rate = HEAT_MAX_WATER_TEMP_UP_RATE,
                cool_max_water_temp_down_rate = COOL_MAX_WATER_TEMP_DOWN_RATE,
                ):
        
        # Set up some variables
        self.fan_max_wattage = np.float32(fan_max_wattage)
        self.heat_max_wattage = np.float32(heat_max_wattage)
        self.cool_max_wattage = np.float32(cool_max_wattage)

        self.fan_max_air_inout_rate = np.float32(fan_max_air_inout_rate)
        self.heat_max_water_temp_up_rate = np.float32(heat_max_water_temp_up_rate)
        self.cool_max_water_temp_down_rate = np.float32(cool_max_water_temp_down_rate)

        # Specify the action_space
        self.action_space = MultiDiscrete([101]*3) # e.g., fan capacity, heating component capacity, cooling component capacity
        
        # Speicfy the observation_space
        self.observation_space = Dict({"InTemp/InHumid/OutTemp/OutHumid/Energy": Box(-100, 100, shape=(5,)),
                                        "Status": Discrete(2),})

    def reset(self, seed=None, demand = None, status = None, renewable = None):
        super().reset(seed=seed) # To enable self.np_random seeding
        rng = self.np_random

        # Start state initialisation
        self.inside_temp = rng.uniform(low = 0, high = 70)
        self.inside_humid = rng.uniform(low = 30, high = 90)
        self.outside_temp = rng.uniform(low = - 10, high = 30)
        self.outside_humid = rng.uniform(low = 30, high = 90)
        
        self.day_night = rng.
        self.energy = 0
        
        self.plant_OK = 0 # 0 means not in the box yet, 1 means OK
        
        observation = {"Energy/CO2/Money/Renewable": np.float32(np.array([self.inside_temp, 
                                                                          self.inside_humid, 
                                                                          self.outside_temp,
                                                                          self.outside_humid,
                                                                          self.energy])),
                        "Status": self.plant_OK}

        info = {}

        return observation, info

    def step(self, action):
        # Interprete action into parameters of mathematical model
        self.fan_air_inout_rate = (action[0] * self.fan_max_air_inout_rate)/100
        self.heat_water_temp_up_rate = (action[1] * self.heat_max_water_temp_up_rate)/100
        self.cool_water_temp_down_rate = (action[2] * self.cool_max_water_temp_down_rate)/100

        self.fan_wattage = (action[0] * self.fan_max_wattage)/100
        self.heat_wattage = (action[1] * self.heat_max_wattage)/100
        self.cool_wattage = (action[2] * self.cool_max_wattage)/100

        # Update observations via model for temp, humididty, VPD
        self.inside_temp += self.fan_air_inout_rate * self.outside_temp \
            + self.heat_water_temp_up_rate * 10 \
            + self.cool_water_temp_down_rate * 10 \
            + self.
        self.inside_humid += self.fan_air_inout_rate * self.outside_humid + 10

